# bfut_FceConverter

```py 
# Author: bfut <https://github.com/bfut> (zlib License)
```

# Description

This notebook is a step-by-step converter for FCE files between FCE3, FCE4, and FCE4M formats.

Handles renaming parts, merging parts, reordering parts, renaming dummies, scaling model size, etc., where applicable.

Also converts texture alpha channel. For FCE3 and FCE4, the source is a TGA file. For FCE4M, the source is an FSH file.

Conversion flows:
```
    FCE4M -> FCE4 -> FCE3
    FCE3  -> FCE4 -> FCE4M
```
Automatically determines the input file format. The target format is set as a parameter. See *Parameterize Notebook* for the how-to.

# Purpose

This notebook can produce a game-ready and finished VIV archive in seconds, avoiding manual chores.

Preview and prepare your work for multiple games simultaneously.

# Required tools

1. `Python 3.9 or later`: Windows or Linux, either is possible. *Note:* Windows users may want to consider using WSL

1. Run from the command-line
    ```sh
    python -m pip install -U pip ipykernel
    ```

1. `fcecodec`: https://github.com/bfut/fcecodec (install Python module)

1. `unvivtool`: https://github.com/bfut/unvivtool (install Python module)

1. Download https://github.com/bfut/PyScripts for

    * `bfut_NfsTgaConverter`

    * `bfut_TextureRotator`

    * `bfut_Tga2Bmp`

1. `ImageMagick`: https://imagemagick.org/script/download.php (install executable) *Note:* only required when working with FSH files

1. `fshtool`: https://github.com/bfut/fshtool/releases (executable)  *Note:* only required when working with FSH files
    ```sh
    # Windows users: extract 'fshtool.exe'
    # WSL/Linux users: extract 'fshtool.c' and run from the command-line
    gcc path/to/fshtool.c -std=c89 -O0 -o fshtool
    ```

# Workload

### Parameterize Notebook

1. `fce_version_target` determines the output FCE format. Set to `3`, `4` or `5` for FCE3, FCE4, or FCE4M, respectively.

Files
1. `fce_path_input` path to the source FCE file (can be FCE3, FCE4, or FCE4M)
1. `fce_path_output` will be path to the output FCE file
1. `fsh_path_input` path to an optional source texture in FSH format. Only used, when source file is in FCE4M format.
1. `car00tga_input` path to an optional source texture in TGA format. Only used, when source file is not in FCE4M format.
1. `viv_target_dir` path to the directory whose files will be encoded to the finished VIV archive. Output FCE and texture file should be written to this directory.
1. `fce_path_colorsource` path to an optional source FCE file whose colors will be copied to the output FCE file, if desired.

Misc
1. `fce_copy_color` copy colors from another FCE file (`True` or `False`)
1. `texture_flip_vertically` sometimes a texture has to be flipped vertically (`True` or `False`)

When target version is FCE3:
1. `convertible_fce3` if True, merge convertible top to body, if False, delete top (:OT) (`True` or `False`)
1. `transparent_windows_fce3` if False, delete interior and high body window triangles are not semi-transparent (`True` or `False`)
1. `rescale_factor_fce3` rescale model size (and dummy positions) by this floating point factor; 1.1 to 1.2 give good results for vanilla FCE4 models

When target version is FCE4M:
1. `fce_path_4Mpartorder_source` path to an optional source FCE4M file whose partnames and partsorder will be mimicked, if desired

When source version is FCE4M:
1. `chopped_roof_fce4m` normal or chopped roof (`True` or `False`)
1. `convertible_fce4m` rename convertible top to :OT; overrides chopped_roof_fce4m option (`True` or `False`)
1. `hood_scoop_fce4m` select hood scoop size (`None` or `"small"` or `"big"`)
1. `pp_wheels_as_wheels_fce4m` rename dummy wheel parts to legitimate wheels (`True` or `False`)

Tools
1. `py_scriptsdir` path to a local copy of the `bfut/PyScripts` repository
1. `fcecodec_dir` path to a local copy of the `bfut/fcecodec` repository
1. `fshtool_exe` path to an `fshtool` executable
1. `unvivtool_script` path to a local copy of `unvivtool_script.py` from the `bfut/unvivtool` repository

In [ ]:
import os
import pathlib
import platform

import fcecodec as fc

fce_version_target = 3

# Files
fce_path_input = f"part.fce"
fce_path_output = f"car.fce"
fsh_path_input = f"part.fsh"
car00tga_input = f"car00.tga"
car00tga_output = f"another/car00.tga"
viv_target_dir = f"path/to/car_viv"
fce_path_colorsource = f"another/car.fce"

# Misc
fce_copy_color = False
texture_flip_vertically = True


# FCE3 target
convertible_fce3 = True
transparent_windows_fce3 = True
rescale_factor_fce3 = 1.0


# FCE4M target
fce_path_4Mpartorder_source = f"path/to/some/part.fce"


# FCE4M source
chopped_roof_fce4m = False
convertible_fce4m = False
hood_scoop_fce4m = None
pp_wheels_as_wheels_fce4m = True


# Tools
py_scriptsdir = f"path/to/PyScripts/"
fcecodec_dir = f"path/to/bfut_fcecodec2/"
fshtool_exe = f"path/to/fshtool"
unvivtool_script = f"path/to/unvivtool_script.py"

### We're set. Run All.

In [ ]:
if str(fce_version_target).lower() == "4m":
    fce_version_target = 5
fce_version_target = int(fce_version_target)
assert fce_version_target in [3, 4, 5]

# 'fce_path_input' is original file and will not be changed
# 'fce_path_current' is variable stand-in for input path in script calls
# 'fce_path_output' is path to output file
fce_path_current = fce_path_input

def GetFceVersion(path):
    with open(path, "rb") as f:
        version = fc.GetFceVersion(f.read(0x2038))
        assert version > 0
        return version

input_fce_version = GetFceVersion(fce_path_current)
print(f"convert {input_fce_version} -> {fce_version_target}")

# Transform parameters
param_translator = {
    None: 0,
    False: 0,
    True: 1,
    "small": 1,
    "big": 2,
    2: 2,
}
convertible_fce3 = param_translator[convertible_fce3]
transparent_windows_fce3 = param_translator[transparent_windows_fce3]
chopped_roof_fce4m = param_translator[chopped_roof_fce4m]
convertible_fce4m = param_translator[convertible_fce4m]
hood_scoop_fce4m = param_translator[hood_scoop_fce4m]

rescale_factor_fce3 = float(rescale_factor_fce3)

Convert from FCE4M to FCE4, if applicable

In [ ]:
current_fce_version = GetFceVersion(fce_path_current)
if current_fce_version == 5 and fce_version_target < 5:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_current}"
    !python "{fcecodec_dir}/scripts/""bfut_MergeParts (FceM to Fce4, keep version).py" "{chopped_roof_fce4m}" "{convertible_fce4m}" "{hood_scoop_fce4m}" "{fce_path_current}" "{fce_path_output}"
    if pp_wheels_as_wheels_fce4m:
        mesh = fc.Mesh()
        with open(fce_path_output, "rb") as f:
            mesh.IoDecode(f.read())
        assert mesh.MValid() is True
        rename_map = {
            ":PPLFwheel": ":HLFW",
            ":PPRFwheel": ":HRFW",
            ":PPLRwheel": ":HLRW",
            ":PPRRwheel": ":HRRW",
        }
        for pid in range(mesh.MNumParts):
            tmp = mesh.PGetName(pid)
            pname = rename_map.get(mesh.PGetName(pid), tmp)
            mesh.PSetName(pid, pname)
        with open(fce_path_output, "wb") as f:
            buf = mesh.IoEncode_Fce4M(False)
            assert fc.ValidateFce(buf) == 1
            f.write(buf)
        del f
        del mesh
    pass
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce4.py" "{fce_path_output}" "{fce_path_output}"

    fce_path_current = fce_path_output

Convert from FCE4 to FCE3, if applicable

In [ ]:
current_fce_version = GetFceVersion(fce_path_current)
if current_fce_version == 4 and fce_version_target < 4:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_current}"
    !python "{fcecodec_dir}/scripts/""bfut_MergeParts (Fce4 to Fce3, keep version).py" "{convertible_fce3}" "{transparent_windows_fce3}" "{fce_path_current}" "{fce_path_output}"
    !echo "DONE bfut_MergeParts (Fce4 to Fce3, keep version).py"
    !python "{fcecodec_dir}/scripts/""bfut_SortPartsToFce3Order (keep fce version).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_RescaleModel.py" "{rescale_factor_fce3}" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_ConvertDummies (to Fce3).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce3.py" "{fce_path_output}" "{fce_path_output}"

    fce_path_current = fce_path_output

Convert from FCE3 to FCE4, if applicable

In [ ]:
current_fce_version = GetFceVersion(fce_path_current)
if current_fce_version == 3 and fce_version_target > 3:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_current}"
    !python "{fcecodec_dir}/scripts/""bfut_ConvertPartnames (Fce3 to Fce4).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_ConvertDummies (Fce3 to Fce4).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce4.py" "{fce_path_output}" "{fce_path_output}"

    fce_path_current = fce_path_output

Convert from FCE4 to FCE4M, if applicable

In [ ]:
current_fce_version = GetFceVersion(fce_path_current)
if current_fce_version == 4 and fce_version_target > 5:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_current}"
    !python "{fcecodec_dir}/scripts/""bfut_ConvertPartnames (Fce4 to Fce4M).py" "{fce_path_output}" "{fce_path_output}"
    if os.path.isfile(fce_path_4Mpartorder_source):
        with open(fce_path_4Mpartorder_source, "rb") as f:
            if fc.ValidateFce(f.read()) == 1:
                pass
                !python "{fcecodec_dir}/scripts/""bfut_MimicPartnamesAndPartsorder.py" "{fce_path_4Mpartorder_source}" "{fce_path_output}"
    pass
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce4M.py" "{fce_path_output}" "{fce_path_output}"

Validate

In [ ]:
with open(fce_path_output, "rb") as f:
    assert fc.ValidateFce(f.read()) == 1

Optionally copy colors

In [ ]:
if fce_copy_color:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_CopyCarColors.py" "{fce_path_colorsource}" "{fce_path_output}"

Convert FSH to TGA, if applicable

In [ ]:
if input_fce_version == 5 and fce_version_target < 5 and len(fsh_path_input) > 0:
    p = pathlib.Path(fsh_path_input)
    car00tga_input = p.with_suffix(".tga")

    !{fshtool_exe} "{fsh_path_input}"
    BMP = (p.parent / p.stem / "0000").with_suffix(".BMP")
    BMPA = (p.parent / p.stem / "0000-a").with_suffix(".BMP")
    !convert "{BMP}" "{BMPA}" -auto-orient -alpha off -compose CopyOpacity -composite "{car00tga_input}"

Convert TGA alpha channel to target version

In [ ]:
if platform.system().lower() == "windows":
    pass
    !copy "{car00tga_input}" "{car00tga_output}"
else:
    pass
    !cp "{car00tga_input}" "{car00tga_output}"

if texture_flip_vertically:
    pass
    !python "{py_scriptsdir}""bfut_TextureRotator/bfut_TextureRotator (flip vertically).py" "{car00tga_output}" "{car00tga_output}"

if fce_version_target == 3:
    if input_fce_version == 4:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (HSto3).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 5:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (Mto3).py" "{car00tga_output}" "{car00tga_output}"
elif fce_version_target == 4:
    if input_fce_version == 3:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (3toHS).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 5:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (MtoHS).py" "{car00tga_output}" "{car00tga_output}"
elif fce_version_target == 5:
    if input_fce_version == 3:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (3toM).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 4:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (HStoM).py" "{car00tga_output}" "{car00tga_output}"

Convert TGA to FSH

In [ ]:
# only applicable for conversion to FCE4M
# a description of the procedure can be found in the "bfut_Tga2Bmp.py" APPENDIX section

Create VIV archive

In [ ]:
!python "{unvivtool_script}" e "{viv_target_dir}"

```py
# Copyright (C) 2023 and later Benjamin Futasz <https://github.com/bfut>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
```